<h3>How to query from opeSearch<h3>

In [15]:
from sentence_transformers import SentenceTransformer
from opensearchpy import OpenSearch

<h3>Connect to Open Search<h3>

In [2]:
#Initialize connection to opensearch
host = 'localhost'
port = 9200
auth = ('admin', 'admin') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    timeout=100
)
#check status
print(client.info())

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'name': 'opensearch-node1', 'cluster_name': 'opensearch-cluster', 'cluster_uuid': '9-DZHZnpTD6C8DMtz6Ys0w', 'version': {'distribution': 'opensearch', 'number': '2.11.1', 'build_type': 'tar', 'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752', 'build_date': '2023-11-29T21:43:10.135035992Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


<h3>Knn Vector Search<h3>

In [4]:
model_name = 'intfloat/e5-large-v2'
model = SentenceTransformer(model_name)

model.safetensors: 100%|██████████| 1.34G/1.34G [04:15<00:00, 5.24MB/s]
pytorch_model.bin: 100%|██████████| 1.34G/1.34G [04:37<00:00, 4.83MB/s]
sentence_bert_config.json: 100%|██████████| 57.0/57.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 4.29MB/s]
tokenizer_config.json: 100%|██████████| 314/314 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 6.23MB/s]
modules.json: 100%|██████████| 387/387 [00:00<?, ?B/s] 


In [26]:
query = "During fiscal 2007, what was the tax benefit recognized by the Company related to the disposition of its German operations?"
query_vector = model.encode("query: "+query)

In [27]:
knn_search_body = {
    "size": 5,  # Number of nearest neighbors to retrieve
    "query": {
        "knn": {
            "vector": {
                "vector": query_vector,
                "k": 1  # Number of nearest neighbors to retrieve
            }
        }
    }
}

# Execute the search
response = client.search(index="wmt_financial_e5_512", body=knn_search_body)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [29]:
text_result = [chunk["_source"]["text"] for chunk in response["hits"]["hits"]]
print(text_result)

['as follows : income from continuing operations before income taxes and minority interest by jurisdiction is as follows ( in millions ) : income from continuing operations before income taxes and minority interest by jurisdiction is as follows ( in millions ) : united states income taxes have not been provided on accumulated but undistributed earnings of its non - u. s. subsidiaries of $ 10. 7 billion and $ 8. 7 billion as of january 31, 2008 and 2007, respectively as the company intends to permanently reinvest these undistributed earn - ings. however, if any portion were to be distributed, the related u. s. tax liability may be reduced by foreign income taxes paid on those earn - ings. determination of the unrecognized deferred tax liability related to these undistributed earnings is not practicable because of the complexities with its hypothetical calculation. the company had foreign net operating loss carryforwards of $ 2. 9 billion at january 31, 2008 and $ 2. 3 billion at january

<h3>Text Search<h3>

In [13]:
text_query = "Second, we are committed throughout the organization to leverage expenses and improve productivity. Our goal remains ery clear: we will grow operating expenses slower than sales and grow operating income faster than sales. By lowering expenses, pssing those savings on to customers, bringing more customers in our doors, and selling more merchandise, we’re reenergizing the “productivity loop” that’s been so vital to Walmart throughout our history. We will do even more to leverage the scale, expenses and expertise of our total company all around the world. We’ll continue to make investments in technology that are clearly driving greater efficiency throughout our company. And we plan to move even quicker and be a more innovative compan"

k = 3

text_search_body = {
    "size": k, 
    "explain": True,
    "query": {
        "match": {
            "text": text_query  
        }
    }
}

response = client.search(index="wmt_financial_e5_512", body=text_search_body)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
hits = response['hits']['hits']
for hit in hits:
    source = hit['_source']
    print(f"Score: {hit['_score']}, Text: {source['text']}")

Score: 115.09949, Text: . more than anyone in our company, our store and club associates take care of our customers and members, and they make incredible contributions to our success. im pleased with the progress that weve already made on diversity and inclusion, and especially with our global womens strategy. we are championing opportunities for women to advance in walmart, and to develop women at all levels throughout the company. driving the productivity loop delivering everyday low prices is premised on everyday low cost. there is no doubt that the pro - ductivity loop is back in walmarts dna. over the past two years, i have personally talked a lot about the productivity loop and its importance. we are very proud that we have leveraged operating expenses for two consecutive years. walmart is operating for less, so we can sell for less and drive increased sales. and we are just getting started. last october, we committed to reducing operating expenses as a percentage of sales by at 